In [1]:
from tweepy import OAuthHandler, Stream, StreamListener
from bs4 import BeautifulSoup
import requests
import json
import io

class StdOutListener(StreamListener):
    def on_data(self, data):
        print(data)
        return True
    def on_error(self, status):
        print(status)

with io.open('config_secret.json') as cred:
    creds = json.load(cred)

    auth = OAuthHandler(creds['CONSUMER_KEY'], creds['CONSUMER_SECRET'])
    auth.set_access_token(creds['OAUTH_TOKEN'], creds['OAUTH_TOKEN_SECRET'])
    stream = Stream(auth, StdOutListener())
#     stream.filter(track=show_names)

In [2]:
from tweepy import OAuthHandler, Stream, StreamListener
from bs4 import BeautifulSoup
import requests, io, json
from tweet_saver import Tweet_Saver
import matplotlib.pyplot as plt
from collections import defaultdict

%matplotlib inline

creds = json.load(io.open('config_secret.json'))

auth = OAuthHandler(creds['CONSUMER_KEY'], creds['CONSUMER_SECRET'])
auth.set_access_token(creds['OAUTH_TOKEN'], creds['OAUTH_TOKEN_SECRET'])

def get_tweets(keywords, file, time_length):       
      
    stream = Stream(auth, Tweet_Saver(file, time_length))
    stream.filter(track=keywords)
    
    print("Done grabbing tweets!")

def graph_tweets(file, keywords):
    
    #tweets_file = open(file, "r")
    #tweets_data = [json.loads(line)['created_at'] for line in open(file, "r")]
    #tweets_data = json.load(io.open(file, "r"))
#     for line in tweets_file:
#         tweets_data.append()
    tweets_data = []
    for tweet in open(file, "r", encoding='utf8'):
        try:
            tweets_data.append(json.loads(tweet))
        except:
            pass
            
    
    tweet_counts = {word:0 for word in keywords}
    for tweet in tweets_data:
        for word in keywords:
            if word in tweet['text']:
                tweet_counts[word] += 1
                    
    output_dict = {}
    for key, value in tweet_counts.items():
        # takes the item (number) and if greater than zero will count associate it to the key
        if value > 0:
            output_dict[key] = value
    
    fig, ax = plt.subplots(figsize=(14,8))
    width = .6
    ax.tick_params(axis='x', labelsize=8)
    ax.tick_params(axis='y', labelsize=10)
    ax.set_xlabel('TV Show', fontsize=15)
    ax.set_ylabel('Number of tweets' , fontsize=15)
    ax.set_title('Top TV Shows', fontsize=15, fontweight='bold')
    plt.bar(range(len(output_dict)), output_dict.values(), width, align='center')
    plt.xticks(range(len(output_dict)), output_dict.keys(), rotation=90)
    plt.xlim([0,len(output_dict)])
    

In [3]:
response = requests.get('http://www.tvguide.com/trending-tonight/')
txt = BeautifulSoup(response.text, 'lxml')

show_names = []
for trends in txt.find_all('a',{'class': 'listings-program-link'}):
    show_names.append(trends.text)

In [4]:
print("Getting tweets")
tweet_file = 'testing2.txt'

get_tweets(show_names, tweet_file, 30)

graph_tweets(tweet_file, show_names)

Getting tweets
Done grabbing tweets!


KeyError: 'text'